# Data Manipulation Language (DML) operations in SQL. 
DML operations are used to manipulate data stored in the database tables. They allow you to add, modify, and remove records, thereby enabling you to interact with and manage the data within your database.

In [ ]:
## Setup Database Connection
import sqlite3
import pandas as pd

conn = sqlite3.connect('fake_data.db')
cursor = conn.cursor()

# Execute the SQL command to list all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

# Fetch all the rows (table names)
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

## Inserting a New Record
You want to insert a new record into a table. For example, you want to insert a new record into the names table. 

In [13]:
cursor.execute('''
               INSERT INTO names(full_name, first_name, last_name, prefix) 
               VALUES ('John Doe', 'John', 'Doe', 'Mr.')
               ''')
conn.commit()

## Inserting Default Values
A table can be defined to take default values for specific columns. You want to insert a row of default values without having to specify those values.

In [16]:
cursor.execute('''
                CREATE TABLE IF NOT EXISTS test_table 
               (id integer default 0,
                foo varchar(10))
                ''')
conn.commit()

## Overriding a Default Value with NULL
You are inserting into a column having a default value, and you want to override that default value by setting the column to NULL.

In [18]:
cursor.execute('''
               INSERT INTO test_table(id, foo) 
               VALUES (null, 'bar')
               ''')
conn.commit()

## Copying Rows from One Table into Another
You want to copy rows from one table to another by using a query. The query may be complex or simple, but ultimately you want the result to be inserted into another table.

In [20]:
cursor.execute('''INSERT INTO test_table(id, foo)
               SELECT first_name, last_name FROM names
               WHERE prefix = 'Mr.'
                ''')
conn.commit()

## Copying a Table Definition
You want to create a new table having the same set of columns as an existing table. 

In [23]:
cursor.execute('''CREATE TABLE IF NOT EXISTS test_table2
               AS
               SELECT * FROM names
               WHERE 1 = 0
                ''')
conn.commit()

## Blocking Inserts to Certain Columns
You want to prevent users, or an errant software application, from inserting values into certain table columns.
Create a view on the table exposing only those columns you want to expose. Then force all inserts to go through that view.

In [24]:
cursor.execute('''CREATE VIEW test_view 
               AS
                SELECT first_name, prefix FROM names
                 ''')
conn.commit()

## Modifying Records in a Table
You want to modify values for some or all rows in a table.


In [ ]:
# Let say you want to increase the salary for every senior position by 10 %
cursor.execute('''UPDATE names
               SET salary = salary * 1.1
               WHERE position = 'Senior'
                ''')
conn.commit()

## Updating When Corresponding Rows Exist
You want to update rows in one table when corresponding rows exist in another.

In [ ]:
# Let say you want to increase the salary for every senior position by 10 % from the health department
cursor.execute('''UPDATE names
               SET salary = salary * 1.1
               WHERE position = (SELECT position FROM healthcare WHERE department = 'Health')
                ''')
conn.commit()

## Updating with Values from Another Table
You want to update rows in one table using values from another.

In [ ]:
cursor.execute('''UPDATE names
               SET salary = AD.salary * 1.1
               FROM addresses AS AD
               WHERE AD.position = names.position
                ''')
conn.commit()

## Merging Records
You want to conditionally insert, update, or delete records in a table depending on whether corresponding records exist. (If a record exists, then update; if not, then insert; if after updating a row fails to meet a certain condition, delete it.) For example, you want to modify table EMP_COMMISSION such that:

In [8]:
# For other databases
('''merge into test_table2 tt
               USING (SELECT * FROM names) n
               ON (tt.id = n.id)
               WHEN MATCHED THEN
               UPDATE SET tt.prefix = Mr.
               DELETE WHERE (first_name = "Ashley")
               WHEN NOT MATCHED THEN
               INSERT (tt.id, tt.full_name, tt.first_name, tt.last_name, tt.prefix)
               VALUES (n.id, n.full_name, n.first_name, n.last_name, n.prefix)
''')

# For the sqlite
cursor.execute('''INSERT OR REPLACE INTO test_table2 (id, full_name, first_name, last_name, prefix)
               SELECT n.id, n.full_name, n.first_name, n.last_name, n.prefix FROM names n;
''')
conn.commit()


## Deleting Referential Integrity Violations
You want to delete records from a table when those records refer to nonexistent records in some other table. For example, some employees are assigned to departments that do not exist. You want to delete those employees.

In [ ]:
cursor.execute('''DELETE FROM emp
               WHERE NOT EXISTS (
               SELECT * FROM dept
               WHERE dept.deptno = emp.deptno
)''')
conn.commit()

## Deleting Duplicate Records
You want to delete duplicate records from a table. Consider the following table, where the duplicate will be judged based on the "num" column.

In [48]:
cursor.execute('''create table test_run (id integer, name varchar(10), num integer);
               ''')
conn.commit()

cursor.execute('''
               insert into test_run (id, name,  num) 
               values (1, 'NAPOLEON', 10),
                (2, 'DYNAMITE', 12),
               (3, 'DYNAMITE', 13),
               (4, 'SHE SELLS', 10),
               (5, 'SEA SHELLS', 14),
               (6, 'SEA SHELLS', 11),
               (7, 'SEA SHELLS', 11)
               ''')
conn.commit()

In [49]:
dupes = pd.read_sql("SELECT * FROM test_run", conn)
display(dupes)

,id,name,num
0,1,NAPOLEON,10
1,2,DYNAMITE,12
2,3,DYNAMITE,13
3,4,SHE SELLS,10
4,5,SEA SHELLS,14
5,6,SEA SHELLS,11
6,7,SEA SHELLS,11


In [50]:
cursor.execute(''' DELETE FROM test_run
WHERE id NOT IN (
    SELECT id
    FROM (
        SELECT id, ROW_NUMBER() OVER (PARTITION BY num ORDER BY name) AS rn
        FROM test_run
    ) AS t
    WHERE rn = 1
)
 ''')
conn.commit()

In [51]:
dupes = pd.read_sql("SELECT DISTINCT id, name, num FROM test_run", conn)
display(dupes)

,id,name,num
0,1,NAPOLEON,10
1,2,DYNAMITE,12
2,3,DYNAMITE,13
3,5,SEA SHELLS,14
4,6,SEA SHELLS,11
